# 3D Image Classification from CT Scans

**Author:** [Hasib Zunair](https://twitter.com/hasibzunair)<br>
**Date created:** 2020/09/23<br>
**Last modified:** 2020/09/23<br>
**Description:** Train a 3D convolutional neural network to predict presence of pneumonia.

## Introduction

This example will show the steps needed to build a 3D convolutional neural network (CNN)
to predict the presence of viral pneumonia in computer tomography (CT) scans. 2D CNNs are
commonly used to process RGB images (3 channels). A 3D CNN is simply the 3D
equivalent: it takes as input a 3D volume or a sequence of 2D frames (e.g. slices in a CT scan),
3D CNNs are a powerful model for learning representations for volumetric data.

## References

- [A survey on Deep Learning Advances on Different 3D DataRepresentations](https://arxiv.org/pdf/1808.01462.pdf)
- [VoxNet: A 3D Convolutional Neural Network for Real-Time Object Recognition](https://www.ri.cmu.edu/pub_files/2015/9/voxnet_maturana_scherer_iros15.pdf)
- [FusionNet: 3D Object Classification Using MultipleData Representations](http://3ddl.cs.princeton.edu/2016/papers/Hegde_Zadeh.pdf)
- [Uniformizing Techniques to Process CT scans with 3D CNNs for Tuberculosis Prediction](https://arxiv.org/abs/2007.13224)

## Setup

In [13]:
!pip install pip --upgrade -q
!pip install nibabel scipy matplotlib tensorflow -q

In [14]:
import os
import zipfile
import numpy as np
import tensorflow as tf
import logging
from tensorflow import keras
from tensorflow.keras import layers

In [15]:
logging.basicConfig(level=logging.INFO)

In [16]:
#
# Load model from storage.
#
import requests
url = "https://koz.s3.amazonaws.com/models/3d_image_classification.h5"
model_file = '3d_image_classification.h5'

filename = os.path.join(os.getcwd(), model_file)
keras.utils.get_file(filename, url)

model = keras.models.load_model(filename)

In [17]:
#
# Load volume data from storage.
#
url = "https://koz.s3.amazonaws.com/data/ct-data.zip"
filename = os.path.join(os.getcwd(), "ct-data.zip")
keras.utils.get_file(filename, url)

# Unzip data in the newly created directory.
with zipfile.ZipFile("ct-data.zip", "r") as z_fp:
    z_fp.extractall("./")

17408000/17402292 [==============================] - 0s 0us/step


In [18]:
!rm -f ct-data.zip

In [19]:
import nibabel as nib
from scipy import ndimage

def read_nifti_file(filepath):
    """Read and load volume"""
    # Read file
    scan = nib.load(filepath)
    # Get raw data
    scan = scan.get_fdata()
    return scan


## Make predictions on a single CT scan

In [20]:
import requests
#
# payload format
# payload = {"data": {"ndarray": X.tolist()} }
#

for i in range(5):
    filename = f'./data/volume{i}.nii.gz'
    v = read_nifti_file(filename)
    #
    # Local prediction.
    prediction = model.predict(np.expand_dims(v, axis=0))[0]
    logging.info(f'Prediction {i} = {prediction}')
    #

    #
    # Prediction via REST.
    #
#     print(f'Serializing and predicting volume {i}')
#     payload = {"data": {"ndarray": v.tolist()} }
#     r = requests.post('http://mymodel-mygroup-ml-mon.apps.ocp.2ff8.sandbox379.opentlc.com/api/v1.0/predictions', 
#                       json = payload)
#     logging.debug(f'response: {r}')
#     j = r.json()['data']['ndarray']
#     print(f'Volume {i} prediction: {j}')
    
#     class_names = ["normal", "abnormal"]
#     print(f'Volume : Prediction for pneumonia is {(prediction > 0.5)}, probability = {100 * prediction[0]:.1f}%, label = {class_names[volume]==1}')


2021-05-19 02:01:19,463 [267] INFO     root: Prediction 0 = [0.8098432]
2021-05-19 02:01:19,751 [267] INFO     root: Prediction 1 = [0.798264]
2021-05-19 02:01:20,064 [267] INFO     root: Prediction 2 = [0.6138762]
2021-05-19 02:01:20,346 [267] INFO     root: Prediction 3 = [0.48922944]
2021-05-19 02:01:20,630 [267] INFO     root: Prediction 4 = [0.9624293]
